# Projeto: Análise de Banco de Dados de Livros (SQL)

## 1. Objetivos do Estudo
O objetivo deste projeto é analisar o banco de dados de um serviço de leitura concorrente para extrair insights que ajudem a formular uma proposta de valor para um novo aplicativo.

As análises focam em:
* Volume de publicações recentes.
* Métricas de engajamento (avaliações e notas).
* Identificação de editoras e autores de destaque.
* Comportamento dos usuários mais ativos.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Configuração da conexão
db_config = {
    'user': 'practicum_student',
    'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7',
    'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
    'port': 5432,
    'db': 'data-analyst-final-project-db'
}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db']
)

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# Função auxiliar para executar queries
def run_query(query):
    return pd.io.sql.read_sql(query, con=engine)

print("Conexão estabelecida com sucesso.")

Conexão estabelecida com sucesso.


## 2. Estudo das Tabelas

Vamos visualizar as primeiras linhas de cada tabela para entender a estrutura dos dados e os relacionamentos.

In [3]:
# Lista das tabelas para inspeção
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']

for table in tables:
    print(f"--- Tabela: {table} ---")
    display(run_query(f"SELECT * FROM {table} LIMIT 10"))
    print("\n")

--- Tabela: books ---


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116




--- Tabela: authors ---


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts




--- Tabela: publishers ---


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
5,6,Aladdin
6,7,Aladdin Paperbacks
7,8,Albin Michel
8,9,Alfred A. Knopf
9,10,Alfred A. Knopf Books for Young Readers




--- Tabela: ratings ---


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4




--- Tabela: reviews ---


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


In [4]:
# Livros Recentes

query_1 = """
SELECT 
    COUNT(book_id) AS books_released_after_2000
FROM 
    books
WHERE 
    publication_date > '2000-01-01';
"""

run_query(query_1)

,books_released_after_2000
0,819


    A consulta retornou a quantidade total de livros no catálogo publicados após o ano 2000. Este número indica o volume de conteúdo "moderno" disponível na base de dados.

In [10]:
# Avaliações e Classificação Média

query_2 = """
SELECT 
    b.book_id,
    b.title,
    COUNT(DISTINCT rv.review_id) AS review_count,
    AVG(rt.rating) AS avg_rating
FROM 
    books b
LEFT JOIN 
    reviews rv ON b.book_id = rv.book_id
LEFT JOIN 
    ratings rt ON b.book_id = rt.book_id
GROUP BY 
    b.book_id, b.title
ORDER BY 
    review_count DESC
LIMIT 10;
"""

run_query(query_2)

,book_id,title,review_count,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,696,The Da Vinci Code (Robert Langdon #2),6,3.830508
2,207,Eat Pray Love,6,3.395833
3,627,The Alchemist,6,3.789474
4,673,The Catcher in the Rye,6,3.825581
5,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
6,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
7,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
8,497,Outlander (Outlander #1),6,4.125000
9,656,The Book Thief,6,4.264151


    Calculamos a contagem de revisões de texto e a nota média para cada título. O uso de `LEFT JOIN` garante que livros sem avaliações também apareçam na lista. `COUNT(DISTINCT ...)` foi usado para evitar duplicatas causadas pela junção de múltiplas tabelas. Esses dados ajudam a identificar os "Best Sellers".

In [7]:
# Editora com mais Livros Relevantes

query_3 = """
SELECT 
    p.publisher,
    COUNT(b.book_id) AS book_count
FROM 
    books b
JOIN 
    publishers p ON b.publisher_id = p.publisher_id
WHERE 
    b.num_pages > 50
GROUP BY 
    p.publisher
ORDER BY 
    book_count DESC
LIMIT 1;
"""

run_query(query_3)

,publisher,book_count
0,Penguin Books,42


    Identificamos a editora líder em volume de publicações "substanciais" (acima de 50 páginas). Esta editora é um parceiro-chave potencial para o novo aplicativo, dada a sua dominância no catálogo.

In [8]:
# Autor com Melhor Avaliação

query_4 = """
SELECT 
    a.author,
    AVG(rt.rating) AS avg_author_rating
FROM 
    books b
JOIN 
    authors a ON b.author_id = a.author_id
JOIN 
    ratings rt ON b.book_id = rt.book_id
WHERE 
    b.book_id IN (
        SELECT book_id 
        FROM ratings 
        GROUP BY book_id 
        HAVING COUNT(rating_id) >= 50
    )
GROUP BY 
    a.author
ORDER BY 
    avg_author_rating DESC
LIMIT 1;
"""

run_query(query_4)

,author,avg_author_rating
0,J.K. Rowling/Mary GrandPré,4.287097


    Para garantir a robustez estatística, filtramos apenas livros com 50 ou mais notas antes de calcular a média do autor. O resultado aponta o autor de maior sucesso crítico e popularidade consistente na plataforma.

In [2]:
# Engajamento de Usuários Ativos

query_5 = """
SELECT 
    AVG(review_count) AS avg_reviews_by_active_users
FROM (
    SELECT 
        username,
        COUNT(review_id) AS review_count
    FROM 
        reviews
    WHERE 
        username IN (
            SELECT username 
            FROM ratings 
            GROUP BY username 
            HAVING COUNT(rating_id) > 50
        )
    GROUP BY 
        username
) AS subquery;
"""

run_query(query_5)

,avg_reviews_by_active_users
0,24.333333


    A análise focou nos "Super Usuários" (aqueles que deram nota para mais de 50 livros). O resultado mostra a média de resenhas de texto escritas por esse grupo. Isso nos ajuda a entender se os usuários que dão muitas notas também tendem a escrever opiniões detalhadas, o que é vital para fomentar a comunidade do novo aplicativo.

# Conclusões 

Com base na análise SQL realizada no banco de dados, chegamos aos seguintes resultados que fundamentam nossa estratégia:

**1. Atualidade do Acervo**
* **Resultado:** Foram identificados **819** livros lançados após 1º de janeiro de 2000.
* **Conclusão:** O catálogo possui uma base sólida de títulos contemporâneos, garantindo que o novo aplicativo não dependa apenas de obras antigas, aumentando a atratividade para o público jovem e moderno.

**2. Obras de Destaque (Avaliações)**
* **Resultado:** O livro com maior número de avaliações é **Twilight (Twilight #1)** com **7** reviews. O livro com a maior nota média é **Harry Potter and the Prisoner of Azkaban** (Nota: **4.4**).
* **Conclusão:** Identificamos os "Best Sellers" para destaque na página inicial e "Joias Ocultas" (alta qualidade, menos reviews) para recomendações personalizadas.

**3. Parceiro Editorial Estratégico**
* **Resultado:** A editora **Penguin Books** lidera o ranking de publicações de livros substanciais (>50 páginas).
* **Conclusão:** Esta editora deve ser o foco primário para parcerias e acordos de licenciamento, pois detém a maior fatia de conteúdo relevante do mercado.

**4. Autoridade Literária**
* **Resultado:** O autor **J.K. Rowling/Mary GrandPré** possui a maior média de avaliações entre aqueles com mais de 50 notas.
* **Conclusão:** Promoções envolvendo este autor tendem a ter alta conversão e satisfação, dada a consistência da qualidade de sua obra percebida pelo público.

**5. Engajamento dos "Super Usuários"**
* **Resultado:** Usuários que avaliaram mais de 50 livros escrevem, em média, **24.3** resenhas de texto.
* **Conclusão:** O público fiel é altamente engajado na produção de conteúdo. O novo app deve priorizar recursos sociais (comentários, feed de reviews) para aproveitar esse comportamento orgânico e reter usuários.